In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf 
from tensorflow import keras 
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [ ]:
train = pd.read_csv(r'../input/avazu-ctr-prediction/train.gz',compression='gzip', nrows =4577464 ,
                                 header=0, sep=',', quotechar='"')
test = pd.read_csv(r'../input/avazu-ctr-prediction/test.gz',compression='gzip', nrows =2000000 ,
                                 header=0, sep=',', quotechar='"')

In [ ]:
#test = pd.read_csv(r'../input/avazu-ctr-prediction/test.gz',compression='gzip',header=0, sep=',', quotechar='"')

In [ ]:
train.head()

In [ ]:
train.shape,test.shape

In [ ]:
train.nunique()

In [ ]:
print(train.isnull().sum()/len(train)*100)
print(test.isnull().sum()/len(test)*100)

In [ ]:
train.click.value_counts(normalize = True)

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(12,12), dpi=800, facecolor='w', edgecolor='k')

In [ ]:
train.plot(kind='box')
plt.show()

In [ ]:
train.groupby(['C1', 'click']).size().unstack().plot(kind='bar', figsize=(12,6))

In [ ]:
train.groupby(['device_conn_type', 'click']).size().unstack().plot(kind='bar', figsize=(12,6))

In [ ]:
train.groupby(['banner_pos', 'click']).size().unstack().plot(kind='bar', figsize=(12,6))

In [ ]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

In [ ]:
leb = defaultdict(LabelEncoder)
train = train.apply(lambda x: leb[x.name].fit_transform(x))
train.head()

In [ ]:
test = test.apply(lambda x: leb[x.name].fit_transform(x))
test.head()

In [ ]:
X= train.drop("click", axis=1)

y= train.click

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
len(X)

In [ ]:
test = scaler.transform(test)
len(test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression()
clf.fit(X_train,y_train)

In [ ]:
lRpredict = clf.predict(X_cv)

In [ ]:
lRpredict_test = clf.predict(test)

In [ ]:
lRpredict_test

In [ ]:
lRpredict1 = clf.predict_proba(X_cv)

In [ ]:
test1 = pd.read_csv(r'../input/avazu-ctr-prediction/test.gz',compression='gzip', nrows =2000000 ,
                                 header=0, sep=',', quotechar='"')

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_cv,lRpredict))

In [ ]:
confusion_matrix(y_cv,lRpredict)

In [ ]:
print(classification_report(y_cv,lRpredict))

In [ ]:
submission = pd.DataFrame({
        "id": test1['id'],
        "click": lRpredict_test
    })
submission.to_csv('click.csv', index=False)
print(submission)